In [ ]:
import os,os.path as osp
import io,gzip
import functools as ft,itertools as it
import pycurl
import numpy as np,pandas as pd

In [ ]:
parent_dir = osp.abspath(osp.join(os.getcwd(),os.pardir))
cfg = osp.join(parent_dir,'genelinks.cfg')
osp.exists(cfg)

In [ ]:
D = {}
with open(cfg,'r') as f:
    lines = [l.strip() for l in f.readlines() if l]
fn = lambda X: X.split('.')[-2]
for k,g in it.groupby(sorted(lines,key=fn),key=fn):
    D[k] = list(g)
D.keys()

In [ ]:
txt = ['23andme','23andme-exome-vcf','ancestry','genes-for-good']
gzd = ['IYG','decodeme','ftdna-illumina']

In [ ]:
def read_23andme(pagebytes):
    cols = ['rsid','chrom','pos','alleles']
    sep = '\t'
    pg = io.StringIO(pagebytes.decode('ascii'))
    args = {
        'comment':'#','sep':sep,'dtype':object,'names':cols
    }
    return pd.read_csv(pg,**args)
def read_ancestry(pagebytes):
    cols = ['rsid','chrom','pos','alleles']
    sep = ','
    args = {
        'skiprows':1,'sep':sep,'dtype':object,'names':cols
    }
    filt = lambda Y: not Y.startswith('#')
    decd = lambda X: X.replace('\t',',',3).replace('\t','')
    junk = pagebytes.decode('ascii')
    fpg = filter(filt,junk.splitlines())
    junk = [decd(x) for x in fpg]
    junk = io.StringIO('\n'.join(junk))
    return pd.read_csv(junk,**args)

In [ ]:
def grab_link(Links,mode='23andme',verbose=False):
    C = pycurl.Curl()
    C.setopt(pycurl.MAXREDIRS,50)
    #buff = io.BytesIO()
    f1 = lambda Y: Y and not Y.startswith('#')
    f2 = lambda X: '-'.join(X.split('/')[-1].split('.')[::2])
    rows = {}
    if verbose:
        print("using mode: {:}".format(mode))
    if mode=='23andme':
        to_df = read_23andme
    elif mode=='ancestry':
        to_df = read_ancestry

    # Who has time for serial anyway?
    if verbose: print("Building multi-object")
    reqs = []
    m = pycurl.CurlMulti()
    for url in Links: 
        buff = io.BytesIO()
        handle = pycurl.Curl()
        handle.setopt(pycurl.URL, url)
        handle.setopt(pycurl.WRITEDATA, buff)
        req = (url, buff, handle)
        m.add_handle(req[2])
        reqs.append(req)
    # Perform multi-request, modified from:
    # https://github.com/rushtongarth/XnaPyCurl/blob/master/src/getters/multi.py
    SELECT_TIMEOUT = 1.0
    num_handles = len(reqs)
    if verbose: print("requesting...")
    while num_handles:
        ret = m.select(SELECT_TIMEOUT)
        if ret == -1:
            continue
        while 1:
            ret, num_handles = m.perform()
            if ret != pycurl.E_CALL_MULTI_PERFORM: 
                break
    if verbose: print("formatting...")
    for u,req,h in reqs:
        if verbose: print(u)
        genes = to_df(req.getvalue())
        rows[f2(u)] = genes
    return rows


In [ ]:
subset = len(D['23andme'])//20
print(subset)

tograb = np.random.choice(D['23andme'],subset)
genes = grab_link(tograb,verbose=True)

In [ ]:
tmp = grab_link(D['ancestry'][:1],'ancestry')['1001-496']